## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [1]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd
from time import sleep
from os import path
from datetime import datetime



In [2]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

##https://ars.ci.transltr.io/ars/api

def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    for child in j['children']:
        print(child['status'])
        error_code = child['code']
        
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                    
                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                #print(val)
                                test_att_values =[]
                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :
                                        test_att_values.append(tx['value'])
                                        dictionary_2[child['actor']['agent']] = test_att_values
                    #else:
                        #dictionary_2[child['actor']['agent']] = [] 
                #else:
                   # dictionary_2[child['actor']['agent']] = []
            
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
                
            
        
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                #dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
        
        
        else:
            nresults = 0
            #dictionary_2[child['actor']['agent']] = []
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error' ': ' + str(error_code)
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' ': ' + str(error_code)
        elif ((child['status'] == 'Unknown') & (nresults == 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Unknown' ': ' + str(error_code)
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return [dictionary, dictionary_2]


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))
    
def make_hyperlink(value):
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [3]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    #print(root)
    for name in files:
        
        if name.endswith((".json")):
            file_read = path.join(root, name)
            dir_name = (os.path.splitext(os.path.basename(root))[0])
            print(file_read)
            
            filename = (os.path.splitext(os.path.basename(file_read))[0])
            print(filename)
            with open(file_read,'r') as inf:
                query = json.load(inf)
                
                kcresult = submit_to_ars(query)
                
                sleep(600)
                
                result_status = retrieve_ars_results(kcresult)
                
        
                dict_workflows[filename] = kcresult
                            
                sleep(100)

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.3_KCNMA1.json
A.3_KCNMA1
https://arax.ncats.io/?source=ARS&id=f6cf13c5-7e1f-4448-86a8-097f6356fddc
Done
Done
ara-arax Done 135
Done
ara-aragorn Done 214
Error
ara-bte Error 0
Done
ara-unsecret Done 99
Done
kp-genetics Done 0
Done
kp-molecular Done 99
Done
ara-explanatory Done 0
Done
ara-improving Done 5
Error
kp-cam Error 0
Error
kp-textmining Error 0
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Done
kp-icees Done 0
Done
ara-robokop Done 30
Done
kp-chp Done 20
Done
kp-cohd ARS Error 0
Done
kp-icees-dili Done 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.8_EGFR_simple.json
A.8_EGFR_simple
https://arax.ncats.io/?source=ARS&id=3bf95503-b235-473e-b12b-eb75a9de7483
Done
Error
ara-aragorn Error 0
Done
ara-arax Done 336
Unknown
ara-bte Unknown 0
Error
ara-unsecret Error 0
Done
kp-genetics Done 0
Done
kp-molecular Done 0
Done
ara-explanatory Done 0
Done
ara-improving ARS Error 0
Erro

https://arax.ncats.io/?source=ARS&id=9323a4fb-ee99-40e5-9ebd-d8698ad76afc
Done
Done
ara-aragorn Done 417
Done
ara-arax Done 183
Error
ara-bte Error 0
Error
ara-unsecret Error 0
Unknown
kp-genetics Unknown 0
Error
kp-molecular Error 0
Done
ara-explanatory Done 0
Done
ara-improving Done 22
Error
kp-cam Error 0
Error
kp-textmining Error 0
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Done
kp-icees ARS Error 0
Done
ara-robokop Done 0
Error
kp-chp Error 0
Error
kp-cohd Error 0
Done
kp-icees-dili ARS Error 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowD/D.3_ssri-heart-disease-one-hop.json
D.3_ssri-heart-disease-one-hop
https://arax.ncats.io/?source=ARS&id=5668ee63-6dc8-4975-9b40-64ba8bfdc216
Done
Done
ara-aragorn Done 28
Done
ara-arax Done 10
Done
ara-bte Done 26247
Done
ara-unsecret Done 5
Done
kp-genetics Done 0
Done
kp-molecular Done 0
Done
ara-explanatory Done 0
Done
ara-improving Done 3
Error
kp-cam Error 0
Done
kp-textmining Done 0
Error
ara-aragorn-e

<br>

### Codes below are for recording messages and generating outout as csv

<br>

In [4]:
dict_workflows

{'A.3_KCNMA1': 'f6cf13c5-7e1f-4448-86a8-097f6356fddc',
 'A.8_EGFR_simple': '3bf95503-b235-473e-b12b-eb75a9de7483',
 'A.0_RHOBTB2_direct': 'a878f12a-30ef-4071-874c-85acc92641f8',
 'A.2_RHOBTB2_twohop': '85f5bc9f-cbf3-4729-9838-22e31a70ae06',
 'A.9_EGFR_advanced': '1aee903b-36b4-4f91-855b-869f17bdc019',
 'A.1_RHOBTB2': 'e3a17bee-a652-4edd-96f7-12b32c68a217',
 'B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi': 'f1402dc2-9e61-41ea-b0e1-231b5a367802',
 'B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi': '09fc2d5c-9083-44b0-afee-0ce399201023',
 'B.4_one-hop-gene-biological-process-or-activity_trapi': 'a47c10e4-9c60-47f2-830a-aa5c50c02561',
 'B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi': 'dce04b92-4576-495d-9072-42b0f81bec04',
 'B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi': '1b8845bf-83b2-4338-a961-3d6240184b02',
 'B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi': '4beece88-5bfb-4b4e-b39e-7d78548a64f4',
 'B.3_DILI-one

In [5]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_ars_results(val)
    
    workflow_result_messages[keys] = result_status

A.3_KCNMA1 f6cf13c5-7e1f-4448-86a8-097f6356fddc
Done
Done
ara-arax Done 135
Done
ara-aragorn Done 214
Error
ara-bte Error 0
Done
ara-unsecret Done 99
Done
kp-genetics Done 0
Done
kp-molecular Done 99
Done
ara-explanatory Done 0
Done
ara-improving Done 5
Error
kp-cam Error 0
Error
kp-textmining Error 0
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Done
kp-icees Done 0
Done
ara-robokop Done 30
Done
kp-chp Done 20
Done
kp-cohd ARS Error 0
Done
kp-icees-dili Done 0
A.8_EGFR_simple 3bf95503-b235-473e-b12b-eb75a9de7483
Done
Error
ara-aragorn Error 0
Done
ara-arax Done 336
Unknown
ara-bte Unknown 0
Error
ara-unsecret Error 0
Done
kp-genetics Done 0
Done
kp-molecular Done 0
Done
ara-explanatory Done 0
Done
ara-improving ARS Error 0
Error
kp-cam Error 0
Unknown
kp-textmining Unknown 0
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Done
kp-icees Done 0
Done
ara-robokop Done 83
Done
kp-chp Done 0
Error
kp-cohd Error 0
Done
kp-icees-dili Done 0
A.0_RHOBTB2_direct a878f12a-30ef

kp-chp Error 0
Error
kp-cohd Error 0
Done
kp-icees-dili ARS Error 0
D.1_parkinsons-crohns 9323a4fb-ee99-40e5-9ebd-d8698ad76afc
Done
Done
ara-aragorn Done 417
Done
ara-arax Done 183
Error
ara-bte Error 0
Error
ara-unsecret Error 0
Unknown
kp-genetics Unknown 0
Error
kp-molecular Error 0
Done
ara-explanatory Done 0
Done
ara-improving Done 22
Error
kp-cam Error 0
Error
kp-textmining Error 0
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Done
kp-icees ARS Error 0
Done
ara-robokop Done 0
Error
kp-chp Error 0
Error
kp-cohd Error 0
Done
kp-icees-dili ARS Error 0
D.3_ssri-heart-disease-one-hop 5668ee63-6dc8-4975-9b40-64ba8bfdc216
Done
Done
ara-aragorn Done 28
Done
ara-arax Done 10
Done
ara-bte Done 26247
Done
ara-unsecret Done 5
Done
kp-genetics Done 0
Done
kp-molecular Done 0
Done
ara-explanatory Done 0
Done
ara-improving Done 3
Error
kp-cam Error 0
Done
kp-textmining Done 0
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Done
kp-icees Done 0
Done
ara-robokop Done 0
Error
k

### Creating dataframe for workflows with PK

<br>

In [6]:
## Convert mesages to a dataframe
col = []
final_dict = defaultdict(list)

for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k][0].items():
        #print(key, value)
#         if key.startswith('kp-'):
#             key_mod = key.replace('kp-','')
#         else:
#             key_mod = key
        
        final_dict[key].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
B.0_Initial_ARS_three-hop_query_results_MESH:D056487
B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi
B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi
B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi
B.3_DILI-one-hop-from-genes
B.4_one-hop-gene-biological-process-or-activity_trapi
C.1_ChemSubstances_related_to_Disease
C.2_ChemSubstances_related_to_GeneSet
D.1_parkinsons-crohns
D.2_ssri-heart-disease
D.3_ssri-heart-disease-one-hop
D.4_tryptophan-kynurenine
D.6_metformin-ferritin


In [7]:
df.replace('ARS Error', 'No Results', regex=True,inplace=True)

### Creating second table with edge attribute source

<br>

In [8]:
final_dict2 = defaultdict(dict)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    count = 0
    
    for key, value in workflow_result_messages[k][1].items():
        final_dict2[k][key] = value

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
B.0_Initial_ARS_three-hop_query_results_MESH:D056487
B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi
B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi
B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi
B.3_DILI-one-hop-from-genes
B.4_one-hop-gene-biological-process-or-activity_trapi
C.1_ChemSubstances_related_to_Disease
C.2_ChemSubstances_related_to_GeneSet
D.1_parkinsons-crohns
D.2_ssri-heart-disease
D.3_ssri-heart-disease-one-hop
D.4_tryptophan-kynurenine
D.6_metformin-ferritin


In [49]:
final_dict2

defaultdict(dict,
            {'A.0_RHOBTB2_direct': {'ara-improving': ['infores:lincs',
               'infores:spoke',
               'infores:improving-agent']},
             'A.1_RHOBTB2': {'ara-arax': ['infores:rtx-kg2',
               'infores:semmeddb',
               'infores:arax'],
              'ara-improving': ['infores:lincs',
               'infores:spoke',
               'infores:improving-agent']},
             'A.2_RHOBTB2_twohop': {'ara-aragorn': ['infores:aragorn-ranker-ara'],
              'ara-arax': ['infores:rtx-kg2', 'infores:dgidb', 'infores:arax'],
              'ara-improving': ['pmid:25184630',
               'infores:civic',
               'infores:spoke',
               'infores:improving-agent']},
             'A.3_KCNMA1': {'ara-arax': ['infores:rtx-kg2',
               'infores:dgidb',
               'infores:arax'],
              'ara-aragorn': ['infores:aragorn-ranker-ara'],
              'ara-unsecret': ['infores:molepro-kp'],
              'kp-molec

In [10]:
final_dictassemble = []
for k, vs in final_dict2.items():
    #print(k,vs)
    for kv, v in vs.items():
        for t in v:
            final_dictassemble.append([k,kv,t])

In [11]:
column_names = ['Workflow', 'ARS-KPs', 'Values']
df2 = pd.DataFrame(final_dictassemble, columns=column_names)
df2 = df2.astype(str)

In [12]:
df2.Values = df2.Values.apply(lambda x: x[2:-2] if ('[' in x) else x)

In [13]:
df2test = df2.groupby(['Workflow','Values'])['ARS-KPs'].agg(list)

In [14]:
df2test = pd.DataFrame(df2test.unstack().T)

In [16]:
df2test

Workflow,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
Values,,,,,,,,,,,,,,,,,,
SEMMED Chemical API,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-bte],NaN,NaN
infores:aragorn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn]
infores:aragorn-ranker-ara,NaN,NaN,[ara-aragorn],"[ara-aragorn, ara-robokop]",[ara-robokop],NaN,[ara-aragorn],NaN,NaN,NaN,"[ara-aragorn, ara-robokop]",NaN,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],NaN,NaN
infores:arax,NaN,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],NaN,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],NaN,[ara-arax],[ara-arax],[ara-arax],NaN,NaN
infores:automat.intact,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn]
infores:bindingdb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-improving],NaN
infores:chembl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-improving],NaN,"[ara-arax, ara-improving]",NaN,NaN
infores:civic,NaN,NaN,[ara-improving],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:cohd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[kp-cohd],NaN,NaN,[kp-cohd],NaN,NaN


In [17]:
#df2test.drop([''], axis=0, inplace=True)

In [ ]:
#df2test.index = df2test.index.map(lambda x: x[2:-2] if ('[' in x) else x)

In [18]:
df2test

Workflow,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
Values,,,,,,,,,,,,,,,,,,
SEMMED Chemical API,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-bte],NaN,NaN
infores:aragorn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn]
infores:aragorn-ranker-ara,NaN,NaN,[ara-aragorn],"[ara-aragorn, ara-robokop]",[ara-robokop],NaN,[ara-aragorn],NaN,NaN,NaN,"[ara-aragorn, ara-robokop]",NaN,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],NaN,NaN
infores:arax,NaN,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],NaN,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],NaN,[ara-arax],[ara-arax],[ara-arax],NaN,NaN
infores:automat.intact,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn]
infores:bindingdb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-improving],NaN
infores:chembl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-improving],NaN,"[ara-arax, ara-improving]",NaN,NaN
infores:civic,NaN,NaN,[ara-improving],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:cohd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[kp-cohd],NaN,NaN,[kp-cohd],NaN,NaN


In [ ]:
#df2.replace([], 'None', regex=True,inplace=True)

In [ ]:
#df2test = df2test.mask(df2test.applymap(type).eq(list) & ~df2test.astype(bool))

In [19]:
df2test = df2test.rename_axis(None)

In [20]:
df2test.columns.name = None

<br>

## Converting the Pk's to hyperlink

<br>

In [21]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [22]:
df.rename({'pk_id': 'pk'}, inplace=True)

In [23]:
df.sort_index(inplace=True)

In [24]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Error: 598,Results: 200,Results: 200,Error: 500,Error: 500,Error: 598,Results: 200,Error: 598,No Results: 200,...,Error: 598,Results: 200,Error: 598,Results: 200,Error: 500,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,Results: 200,...,Error: 400,Results: 200,Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598
ara-bte,Unknown: 503,Unknown: 503,Unknown: 503,Error: 598,Unknown: 503,Unknown: 503,Error: 502,Unknown: 503,Error: 502,Error: 502,...,Error: 502,Error: 502,Error: 502,No Results: 200,Error: 502,Error: 502,Error: 598,Results: 200,Error: 400,Error: 598
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,No Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-robokop,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 422,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-unsecret,No Results: 200,No Results: 200,No Results: 200,Results: 200,Error: 504,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,Error: 504,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Results: 200,No Results: 200,Results: 200
kp-cam,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,...,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502
kp-chp,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,No Results: 200,...,No Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,Error: 598,No Results: 200,Error: 598,No Results: 200,No Results: 200


<br>

### Highlight the cells In Excel

<br>

In [25]:
def highlight(v):
    if v.startswith('Results'):
        return 'background-color: %s' % 'green'
    elif v.startswith('Error'):
        return 'background-color: %s' % 'red'
    elif v.startswith('No Results'):
        return 'background-color: %s' % 'yellow'
    elif v.startswith('ARS Error'):
        return 'background-color: %s' % 'blue'
    elif v.startswith('Unknown'):
        return 'background-color: %s' % 'magneta'
    else:
        return

In [26]:
styled = df.style.applymap(highlight)

In [27]:
styled

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Error: 598,Results: 200,Results: 200,Error: 500,Error: 500,Error: 598,Results: 200,Error: 598,No Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,Results: 200,Error: 598,Results: 200,Error: 500,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,Results: 200,No Results: 200,Results: 200,Results: 200,Error: 400,Results: 200,Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598
ara-bte,Unknown: 503,Unknown: 503,Unknown: 503,Error: 598,Unknown: 503,Unknown: 503,Error: 502,Unknown: 503,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,No Results: 200,Error: 502,Error: 502,Error: 598,Results: 200,Error: 400,Error: 598
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,No Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-robokop,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 500,No Results: 200,No Results: 200,Error: 422,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-unsecret,No Results: 200,No Results: 200,No Results: 200,Results: 200,Error: 504,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Results: 200,No Results: 200,Results: 200
kp-cam,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502
kp-chp,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,No Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,Erro

In [ ]:
#import xlsxwriter

In [ ]:
#import xlwt

In [28]:
date = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
wks_name = 'Workflow Progress Tracker_' + date

In [29]:
wks_name

'Workflow Progress Tracker_2021_09_10-08_18_51_AM'

In [ ]:
#wks_name = 'Workflow Progress Tracker_2021_09_03-01_13_17_AM'

In [ ]:
#wks_name = 'Workflow Progress Tracker_2021_08_26-03_56_45_AM'

In [ ]:
#writer = pd.ExcelWriter('/Users/priyash/Documents/GitHub/minihackathons/Notebooks/' + wks_name + '.xlsx', engine='xlsxwriter')

In [30]:
styled.to_excel('/Users/priyash/Documents/GitHub/minihackathons/Notebooks/' + wks_name + '.xlsx')

#styled.to_excel(writer, sheet_name='progress_tracker')
#df2.to_excel(writer, sheet_name='edge_attribute')

/usr/local/lib/python3.9/site-packages/pandas/io/formats/excel.py:375: CSSWarning: Unhandled color format: 'magneta'
  warnings.warn(f"Unhandled color format: {repr(val)}", CSSWarning)


In [ ]:
### Save as excel sheet
#df.to_excel('ara_worklow_progress_tracker_Aug5.xlsx', sheet_name = 'Progress_Tracker')

In [ ]:
### save all the ids for ara submission
#with open('ara_worklow_IDs_Aug9.txt', 'w') as f:
#    print(dict_workflows, file=f)

<br>

### Pushing dataframe to excel sheet on google drive

<br>


**Here I am using the google drive API to push the daatframe into an axcel sheet 
Every individula has the unique credential file that they need to create for google drive API -- 
"araxworkflowprogresstesting-2632632db8be.json" -- is the credential used from my drive. place this json file where
the ReadAndRunAllWorkFLows.ipynb will be. NB: i have removed my credntial file for privacy reasons. Always remove
the json file before making committs to the repo. To use googe Drive API follow: https://towardsdatascience.com/how-to-manage-files-in-google-drive-with-python-d26471d91ecd**

<br>

In [31]:
styled

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Error: 598,Results: 200,Results: 200,Error: 500,Error: 500,Error: 598,Results: 200,Error: 598,No Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,Results: 200,Error: 598,Results: 200,Error: 500,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,Results: 200,No Results: 200,Results: 200,Results: 200,Error: 400,Results: 200,Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598
ara-bte,Unknown: 503,Unknown: 503,Unknown: 503,Error: 598,Unknown: 503,Unknown: 503,Error: 502,Unknown: 503,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,No Results: 200,Error: 502,Error: 502,Error: 598,Results: 200,Error: 400,Error: 598
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,No Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-robokop,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 500,No Results: 200,No Results: 200,Error: 422,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-unsecret,No Results: 200,No Results: 200,No Results: 200,Results: 200,Error: 504,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Results: 200,No Results: 200,Results: 200
kp-cam,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502
kp-chp,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,No Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,Erro

In [32]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Error: 598,Results: 200,Results: 200,Error: 500,Error: 500,Error: 598,Results: 200,Error: 598,No Results: 200,...,Error: 598,Results: 200,Error: 598,Results: 200,Error: 500,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,Results: 200,...,Error: 400,Results: 200,Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598
ara-bte,Unknown: 503,Unknown: 503,Unknown: 503,Error: 598,Unknown: 503,Unknown: 503,Error: 502,Unknown: 503,Error: 502,Error: 502,...,Error: 502,Error: 502,Error: 502,No Results: 200,Error: 502,Error: 502,Error: 598,Results: 200,Error: 400,Error: 598
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,No Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-robokop,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 422,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-unsecret,No Results: 200,No Results: 200,No Results: 200,Results: 200,Error: 504,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,Error: 504,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Results: 200,No Results: 200,Results: 200
kp-cam,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,...,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502
kp-chp,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,No Results: 200,...,No Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,Error: 598,No Results: 200,Error: 598,No Results: 200,No Results: 200


<br>

## Google Drive API

**Push the dataframe to a google sheet via google drive API and then format the google spread sheet to add hyperlink to pk's and color the cells

<br>

### Push Dataframe 1

<br>

In [33]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

In [34]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [35]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet 'Workflow Progress Tracker_2021_09_10-08_18_51_AM' id:1859895571>

In [36]:
spreadsheet_key

'1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'

In [37]:
import gspread
import pandas as pd
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *

In [38]:
pwd

'/Users/priyash/Documents/GitHub/minihackathons/Notebooks'

In [39]:
gc = gspread.service_account(filename='/Users/priyash/Documents/GitHub/minihackathons/Notebooks/araxworkflowprogresstesting-2632632db8be.json')
wksh = gc.open("workflow_progress_tracker")
sh = wksh.worksheet(wks_name)

In [40]:
sh

<Worksheet 'Workflow Progress Tracker_2021_09_10-08_18_51_AM' id:1859895571>

In [41]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:{}18', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(1,0,0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{}]}

In [42]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:{}18', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.5, 0.0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1859895571,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 1,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 1}}}}}}},
  {},
  {}]}

In [43]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:{}18', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['No Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0.75, 0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1859895571,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 1,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.49803922}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1859895571,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 1,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'r

In [44]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:{}18', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['ARS Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.75, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1859895571,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 1,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'No Results'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'green': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 0.7490196,
         'green': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1859895571,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 1,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bo

In [45]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:{}18', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Unknown']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1859895571,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 1,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'ARS Error'}]},
      'format': {'backgroundColor': {'green': 0.7490196, 'blue': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.7490196,
         'blue': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1859895571,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 1,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'No Results'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'green': 0.7490196},
    

In [46]:
sh

<Worksheet 'Workflow Progress Tracker_2021_09_10-08_18_51_AM' id:1859895571>

In [47]:
# Select a range
cell_list = sh.range('B17:Y19')

# Update in batch
sh.update_cells(cell_list,value_input_option='USER_ENTERED')

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'updatedRange': "'Workflow Progress Tracker_2021_09_10-08_18_51_AM'!B17:Y19",
 'updatedRows': 3,
 'updatedColumns': 24,
 'updatedCells': 72}

### Push Dataframe 2

<br>

In [48]:
wks2 = 'edge_attribute_source_' + date
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df2test, spreadsheet_key, wks2, credentials=credentials, row_names=True)

<Worksheet 'edge_attribute_source_2021_09_10-08_18_51_AM' id:69738884>